# Chapter09 独立同一分布

- 確率変数が互いに独立であるとは、
  - 確率変数がほかの確率変数に影響を及ぼさないこと。
- 独立同一分布（independently and identically distributed, i.i.d.）
  - お互いに独立で、さらにそれぞれが同じ確率分布に従う多次元確率変数のこと
  - それらが従う確率分を$F$とすると、
    - $X_1, X_2, ..., X_n \ims^{iid} F$
  - と表される。
  - 「同じ条件下で行われる実験や観測を複数回繰り返すことでデータを得る」ということを、数学の言葉で表したもの
  - 統計解析においてもっとも基本的でかつ重要な条件設定。

In [20]:
# ライブラリーの準備
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy import stats
import matplotlib.animation as ani
from IPython.display import HTML

plt.style.use('seaborn')
rcParams['font.family'] = 'IPAexgothic'
np.random.seed(0)
%precision 3
%matplotlib inline

In [21]:
# 離散型確率変数の関する関数
linestyles = ['-', '--', '-.']

def E(XY, g):
    """2次元の離散型確率変数の期待値"""
    x_set, y_set, f_XY = XY
    return np.sum([
        g(x_i, y_j) * f_XY(x_i, y_j)
        for x_i in x_set for y_j in y_set
    ])

def Cov(XY):
    """2次元の離散型確率変数の共分散"""
    x_set, y_set, f_XY = XY
    mean_X = E(XY, lambda x, y: x)
    mean_Y = E(XY, lambda x, y: y)
    return np.sum([
        (x_i-mean_X) * (y_j-mean_Y) * f_XY(x_i, y_j)
        for x_i in x_set for y_j in y_set
    ])

def f_X(x):
    """確率変数Xの周辺分布"""
    return np.sum([
        f_XY(x, y_k) for y_k in y_set
    ])

def f_Y(y):
    """確率変数Yの周辺分布"""
    return np.sum([
        f_XY(x_k, y) for x_k in x_set
    ])

## 9.1 | 独立性

### 9.1.1 独立性の定義

- 確率変数の独立性（independence）とは
  - 2つ以上の確率変数が互いに影響を及ぼさず無関係であることを表す概念

2次元確率変数$(X, Y)$の場合、次のような関係が成り立つことを、**独立である**という。

$$
f_{X, Y}(x, y) = f_X(x)f_Y(y)
$$

すなわち、確率変数が独立なとき、同時確率は周辺確率の積で書くことができる。

#### 独立性

n個の確率変数 $X_1, X_2, ..., X_n$ が、

$$
f_{X_1, X_2, ..., X_n}(x_1, x_2, ..., x_n) = 
f_{X_1}(x_1)f_{X_2}(x_2)...f_{X_n}(x_n)
$$

を満たすとき、$X_1, X_2, ..., X_n$は互いに独立であるという。

ただし、関数$f$は、

- 離散型であれば、確率関数
- 連続型であれば、密度関数

を表す。

### 9.1.2 独立性と無相関性

- 共分散や相関係数が「0」のとき、
  - 2つの確率変数の間に、相関性がないことを表している。
  - 相関性がないとは、直線的な関係がないこと。

- 無相関性より独立性のほうが強い概念
  - 2つの確率変数$XとY$が「独立なとき」、$XとY$は「無相関」。
  - 2つの確率変数$XとY$が「無相関なとき」$XとY$は「必ずしも独立とはなっていない」。
    - 2つの確率変数の間に直線的な関係はないものの、影響を及ぼし合う場合がある。

#### 2つの確率変数$XとY$が「独立なとき」、$XとY$は「無相関」

In [22]:
x_set = np.array([1, 2, 3, 4, 5, 6])
y_set = np.array([1, 2, 3, 4, 5, 6])

def f_XY(x, y):
    """2つのいかさまサイコロの期待値"""
    if x in x_set and y in y_set:
        return x * y / 441
    else:
        return 0

# 確率変数の定義
XY = [x_set, y_set, f_XY]

In [23]:
Cov(XY)

-0.000

独立性の定義はすべての$x, y$に対して、

$$
f_{X, Y}(x, y) = f_X(x)f_Y(y)
$$

が成り立つ。

ここで、$x = 1, y = 1$を代入して、

$$
f_{X, Y}(1, 1) = f_X(1)f_Y(1)
$$

が成り立つか確認する。

In [24]:
f_XY(1, 1), f_X(1) * f_Y(1)

(0.002, 0.002)

等式が成り立っている、$XとY$は独立。

#### 2つの確率変数$XとY$が「無相関なとき」$XとY$は「必ずしも独立とはなっていない」

- 無相関なのに、独立でない例
  - とりうる値の組み合わせが$\{(0, 0), (1, 1), (1, -1)\}$で、
  - それぞれの確率が等しい2次元確率変数$(X, Y)$を使う。

$$
f_{XY}(x, y) = 
\begin{cases}
\begin{align*}
1/3 ~~~ &(~(x, y) \in \{(0, 0), (1, 1), (1, -1)\}~)
\\
0 ~~~~~ &(otherwise)
\end{align*}
\end{cases}
$$

In [25]:
x_set = np.array([0, 1])
y_set = np.array([-1, 0, 1])

def f_XY(x, y):
    if (x, y) in [(0, 0), (1, 1), (1, -1)]:
        return 1 / 3
    else:
        return 0

XY = [x_set, y_set, f_XY]

In [26]:
Cov(XY)

0.000

独立性の定義はすべての$x, y$に対して、

$$
f_{X, Y}(x, y) = f_X(x)f_Y(y)
$$

が成り立つ。

ここで、$x = 0, y = 0$を代入して、

$$
f_{X, Y}(0, 0) = f_X(0)f_Y(0)
$$

が成り立つか確認する。

In [27]:
f_XY(0, 0), f_X(0) * f_Y(0)

(0.333, 0.111)

等式が成り立っていないため、$XとY$は独立ではない。

## | 9.2 和の分布

- 和の分布
  - 互いに独立に同一の確率分布に従う「確率変数 $X_1, X_2, ..., X_n$」の<br>
    「和 $\sum_{i=1}^{n} X_i = X_1 + X_2 + ... + X_n$」が従う確率分布のこと

### 確率変数の和の期待値

和の分布の期待値は、それぞれの確率変数の期待の和で計算できる。

確率変数 $X_1, X_2, ..., X_n$ について、

$$
E(X_1 + X_2 + ... + X_n) = E(X_1) + E(X_2) + ... + E(X_n)
$$

が成り立つ。

※この公式は確率変数が独立でなくても成り立つ。

### 確率変数の和の分散

和の分布の分散も、それぞれの確率変数の分散の和で計算できる。

確率変数 $X_1, X_2, ..., X_n$ が互いに独立ならば、

$$
V(X_1 + X_2 + ... + X_n) = V(X_1) + V(X_2) + ... + V(X_n)
$$

が成り立つ。

### 9.2.1 正規分布の和の分布